### Coursera Capstone Project
#### THU LE
This notebook includes data analysis and machine learning techniques using Python for Capstone Project


#### Import libraries

In [2]:

import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests

#### TASK 1: CREATE DATAFRAME OF POSTAL CODE, BOROUGHS AND NEIGHBORHOODS IN TORONTO

#### Create Soup object

In [3]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
soup = BeautifulSoup(data,'html.parser')



Iritate through each row of the table found from the Soup object\
Extract text from rows to append to the dictionary 'cell' with corresponding keys\
Append dictionaries to final list

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

print(table_contents[0])


{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}


In [85]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [73]:
print('The dataframe has {} boroughs and {} neighborhoods'.format(len(df['Borough'].unique()),df.shape[0]))


The dataframe has 15 boroughs and 103 neighborhoods


### TASK 2: Get the geographical coordinates of the neighborhoods

Google geocoder package takes a long time to run so I use the available coordinate file

In [88]:
geo_coords = pd.read_csv(r'C:\Users\user\Downloads\Geospatial_Coordinates.csv')
geo_coords
df=df.merge(geo_coords, left_on ='PostalCode', right_on ='Postal Code').drop('Postal Code', axis =1)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Task 3: Explore and cluster the neighborhoods in Toronto

I decide to cluster only neighborhoods in Downtown Toronto


In [89]:
downtown = df[df['Borough']=='Downtown Toronto'].reset_index(drop=True)
downtown.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


Define Foursquare credentials and version

In [90]:
CLIENT_ID = 'VNXGPRTGYKVCFEQE23DCKU11I3ETI4OFTQLWMTA2VMTFDMZ0' 
CLIENT_SECRET = 'RKK5YIFUL0JJBUIVD5M4OIUQ1MNDYVZ4YPOR4OXSXEIGNHDR'
VERSION = '20210505' 
LIMIT = 100 


Create function to get venues of all neighborhoods in Downtown Toronto

In [91]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the function to have dataframe of venues in radius of 500 for each neighorhood in downtown

In [92]:
downtown_venues = getNearbyVenues(downtown['Neighborhood'], downtown['Latitude'], downtown['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [93]:
print('There are {} unique categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 200 unique categories.


Create dataframe with frequency of categories for each neighborhood

In [94]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.016667,0.000000,0.000000,0.00,0.00,0.016667,0.00000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.016129,0.000000,0.000000,0.00,0.00,0.016129,0.00000,0.000000,0.016129,0.016129
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.013158,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013158,0.000000,...,0.013158,0.013158,0.013158,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.026316
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.00000,0.000000,0.010000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.01000,0.010000,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.047619,0.00000,0.047619,0.015873,0.000000


#### Run k-means to cluster the neighborhood into 4 clusters.

In [114]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 2, 0, 0, 0, 0, 0, 0])

Dataframe that display the top 10 venues for each neighborhood.

In [115]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Pharmacy,Cheese Shop,Seafood Restaurant,Farmers Market,Creperie
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Rental Car Location,Sculpture Garden,Boat or Ferry,Airport,Airport Gate
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Bubble Tea Shop,Salad Place,Japanese Restaurant,Spa,Sushi Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Baby Store,Candy Store,Restaurant,Nightclub,Coffee Shop
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Pub


Dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [117]:

downtown_merged 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Chocolate Shop,Mexican Restaurant,Brewery
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Italian Restaurant,Fast Food Restaurant,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Cosmetics Shop,Cocktail Bar,Lingerie Store,Park,Department Store,Restaurant,Creperie
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Pharmacy,Cheese Shop,Seafood Restaurant,Farmers Market,Creperie
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Bubble Tea Shop,Salad Place,Japanese Restaurant,Spa,Sushi Restaurant
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Baby Store,Candy Store,Restaurant,Nightclub,Coffee Shop
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,Clothing Store,Gym,American Restaurant,Hotel,Bookstore
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Hotel,Brewery,Scenic Lookout,Sporting Goods Shop,Pizza Place,Restaurant,Italian Restaurant
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Salad Place,Restaurant,Breakfast Spot,Beer Bar
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Cocktail Bar,Deli / Bodega,Seafood Restaurant,American Restaurant


Visualize the resulting clusters

In [118]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [119]:
import folium
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

EXAMINE CLUSTERS

In [121]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Chocolate Shop,Mexican Restaurant,Brewery
1,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Italian Restaurant,Fast Food Restaurant,Pizza Place
2,St. James Town,0,Coffee Shop,Café,Gastropub,Cosmetics Shop,Cocktail Bar,Lingerie Store,Park,Department Store,Restaurant,Creperie
3,Berczy Park,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Pharmacy,Cheese Shop,Seafood Restaurant,Farmers Market,Creperie
4,Central Bay Street,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Bubble Tea Shop,Salad Place,Japanese Restaurant,Spa,Sushi Restaurant
6,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,Clothing Store,Gym,American Restaurant,Hotel,Bookstore
7,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Aquarium,Café,Hotel,Brewery,Scenic Lookout,Sporting Goods Shop,Pizza Place,Restaurant,Italian Restaurant
8,"Toronto Dominion Centre, Design Exchange",0,Coffee Shop,Hotel,Café,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Salad Place,Restaurant,Breakfast Spot,Beer Bar
9,"Commerce Court, Victoria Hotel",0,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Cocktail Bar,Deli / Bodega,Seafood Restaurant,American Restaurant
11,"Kensington Market, Chinatown, Grange Park",0,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Gaming Cafe,Mexican Restaurant,Dessert Shop,Park,Farmers Market


In [122]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Rosedale,1,Park,Playground,Trail,Adult Boutique,Museum,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


In [123]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Christie,2,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Baby Store,Candy Store,Restaurant,Nightclub,Coffee Shop


In [124]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"CN Tower, King and Spadina, Railway Lands, Har...",3,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Rental Car Location,Sculpture Garden,Boat or Ferry,Airport,Airport Gate


In [125]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"University of Toronto, Harbord",4,Café,Bakery,Bar,Japanese Restaurant,Bookstore,College Arts Building,Beer Bar,Beer Store,Sandwich Place,Restaurant
